<a href="https://colab.research.google.com/github/arnottrcaiado/pet-centrocar-2021/blob/main/pdf_clientes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Projeto PET - Centrocar
# Tratamento de dados de Clientes - Conversao de PDF para CSV

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving grupos (1).csv to grupos (1).csv


In [ ]:
!ls -la

total 1380
drwxr-xr-x 1 root root    4096 Nov 29 12:18  .
drwxr-xr-x 1 root root    4096 Nov 29 12:10  ..
-rw-r--r-- 1 root root 1394877 Nov 29 12:19 'CLIENTES (1).PDF'
drwxr-xr-x 4 root root    4096 Nov 18 14:35  .config
drwxr-xr-x 1 root root    4096 Nov 18 14:36  sample_data


In [ ]:
import pandas as pd

In [ ]:
import io

In [ ]:
!ls -la

total 5404
drwxr-xr-x 1 root root    4096 Nov 29 14:37  .
drwxr-xr-x 1 root root    4096 Nov 29 12:10  ..
-rw-r--r-- 1 root root 1394877 Nov 29 12:19 'CLIENTES (1).PDF'
-rw-r--r-- 1 root root 1815844 Nov 29 14:36  Clientes_Codigo.csv
drwxr-xr-x 4 root root    4096 Nov 18 14:35  .config
-rw-r--r-- 1 root root 2300398 Nov 29 14:37 'grupos (1).csv'
drwxr-xr-x 1 root root    4096 Nov 18 14:36  sample_data


In [ ]:
!pip3 install fitz

In [ ]:
!pip install pymupdf

In [ ]:
# importa biblioteca
import fitz

# faz a leitura
with fitz.open( 'CLIENTES (1).PDF' ) as pdf:
    texto = ""
    for pagina in pdf:
        texto += pagina.getText()
texto

In [ ]:
linhas = texto.splitlines()

In [ ]:
colunas =['Nome','Codigo','Data Nasc','Bairro','Cidade']
dados =[]

In [ ]:
def trataLinha( linha ):
  nome =''
  codigo = ''
  data = ''
  bairro= ''
  cidade =''
  nlinha = i.split("   ")
  for j in range(0,len(nlinha)):
    nlinha[j] = nlinha[j].lstrip()

  pos =0
  for j in range(1, len(nlinha)) :
    if len(nlinha[j]) > 0 :
      pos= pos+1
      if pos == 1:
        codigo = nlinha[j]
        if len(codigo) > 7 :
          ncodigo = codigo[7:-1]
          codigo = codigo[:7]
          if len(ncodigo) >10 :
            data = ncodigo[:11]
            if len(ncodigo) >10 :
              bairro = ncodigo[12:]
              pos=5
      if pos == 2:
        data = nlinha[j]
        if data[0] not in '0123456789':
          data =''
          bairro = nlinha[j]
          pos=5
      if pos == 3: # bairro
        bairro = nlinha[j]

  nome = nlinha[0]
  cidade = nlinha[-1]
  if bairro == cidade :
    bairro =''
  return nome, codigo, data, bairro, cidade

In [ ]:
for i in linhas[7:-1] :
  dados.append( trataLinha(i))

In [ ]:
df = pd.DataFrame(dados, columns = colunas)


In [ ]:
df.to_csv( 'Clientes_Codigo.csv',sep=';',index=False)

In [ ]:
dfn=pd.read_csv('grupos (1).csv')

In [ ]:
!ls -la

total 3156
drwxr-xr-x 1 root root    4096 Nov 29 14:10  .
drwxr-xr-x 1 root root    4096 Nov 29 12:10  ..
-rw-r--r-- 1 root root 1394877 Nov 29 12:19 'CLIENTES (1).PDF'
-rw-r--r-- 1 root root 1815844 Nov 29 14:10  Clientes_Codigo.csv
drwxr-xr-x 4 root root    4096 Nov 18 14:35  .config
drwxr-xr-x 1 root root    4096 Nov 18 14:36  sample_data


In [ ]:
df['Sexo']=''

In [ ]:
import requests, json

In [ ]:
for i in range( len(df )):
  # chamar a api que busca sexo a partir de um nome
  resposta = requests.get( 'https://centrocarpet.pythonanywhere.com/buscasexo?nome='+df.loc[i,'Nome'])
  df.loc[i,'Sexo'] = resposta.json()['Sexo']
  # print( df.loc[i,'Consultor']+" Sexo:"+resposta.json()['Sexo'])

In [ ]:
for i in range( len(df )):
  # chamar a api que busca sexo a partir de um nome
  resposta = procNome( df.loc[i,'Nome'])
  df.loc[i,'Sexo'] = resposta
  print( df.loc[i])
  # print( df.loc[i,'Consultor']+" Sexo:"+resposta.json()['Sexo'])

In [ ]:
def procNome( nome ):
    if nome != None :
        lnome = nome.split()
        nome = lnome[0]
        nome = nome.upper()

        busca = dfn.loc[dfn['name']== nome]
        if len(busca) > 0 :
            nomeEncontrado = busca['name'].to_list()
            sexEncontrado = busca['classification'].to_list()
            # return json.dumps({"Nome": nomeEncontrado[0], "Sexo" : sexEncontrado[0] })
            return sexEncontrado[0]
        else:
            sexo = procNomes( nome )
            # return json.dumps({"Nome" : nome, "Sexo": sexo })
            return sexo

# -------------------------------------------------------------------------------------------
# funcao para percorrer o dataframe procurando os nomes alterativos
# chamada quando o nome principal, indexado, não é localizado
def procNomes( nome ) :
    for i in range(len(dfn)) :
      nam = dfn['names'][i].split(sep='|')
      for j in range(len(nam)):
        if nome == nam[j] :
            sex = dfn['classification'][i]
            return str(sex)
    return ""


In [ ]:
df.to_csv( 'Clientes_CodNomeSexo.csv',sep=';',index=False)

In [ ]:
from google.colab import drive
drive.mount ('/drive')

Mounted at /drive


In [ ]:
df.to_csv('/drive/MyDrive/Clientes_CodNomeSexo.csv',sep=';',index=False)

In [ ]:
df1 = df[['Codigo','Data Nasc','Bairro','Cidade','Sexo']]


In [ ]:
df1.to_csv('/drive/MyDrive/Clientes_CodSexo.csv',sep=';',index=False)